In [31]:
#!python --version

In [32]:
#!pip install -q -U pip
#!pip install -q numpy
#!pip install -q pandas
#!pip install -q ckiptagger
#!pip install -q tqdm
#!pip install -q tensorflow==1.14.0
#!pip install -q ipywidgets
#!pip install -q gast==0.2.2

In [33]:
from ckiptagger import WS, POS
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [34]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [35]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [36]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [37]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [38]:
ws = WS('../data/')
pos = POS('../data/')

In [39]:
train_title_cuts =[]
word_s_train = ws(
    list(train_titles.values()),
    sentence_segmentation=True,
    segment_delimiter_set = {",", "。", ":", "?", "!", ";"}
    )
word_p_train = pos(word_s_train)

In [40]:
for x, y in zip(word_s_train, word_p_train):
    train_title_cuts.append(list(zip(x, y)))

In [41]:
test_title_cuts =[]
word_s_test = ws(
    list(test_titles.values()),
    sentence_segmentation=True,
    segment_delimiter_set = {",", "。", ":", "?", "!", ";"}
    )
word_p_test = pos(word_s_test)

for x, y in zip(word_s_test, word_p_test):
    test_title_cuts.append(list(zip(x, y)))

In [42]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

In [43]:
words_train = []
words_test = []
words = []
for key in word_s_train:
    words_train = words_train + key
    
for key in word_s_test:
    words_test = words_test + key
    
words = list(set(words_train + words_test))

# Bag of Words (BOW)

In [44]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
word2index = dict(zip(words,range(len(words))))
index2word = dict(zip(range(len(words)),words))

In [45]:
w_index = word2index['溫暖']
w_index

7826

In [46]:
index2word[w_index]

'溫暖'

In [47]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

In [48]:
def get_bow_vector(pairs, word2index):
    vector = np.zeros(len(word2index.items()))
    for x,y in pairs:
        vector[word2index[x]] = 1
    return vector

In [49]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [50]:
pos_analysis = {}
for pairs in train_title_cuts:
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['朱嘯虎', '泰倫盧成', '巔峰賽', '李盈瑩', '詹皇', '英特爾', '嘉年華', '艾科麥佛鯊', '斐濟', '裴擒虎', '白起', '奧運', '男乒', '丘誠', '威少', '靜懿', '金英權', '郭懿', '夢琪', '劉代全', '鄭眼', '埃爾克森', '足協杯', '姚勁波', '斯威', '多浪', '藍星', '李', '卡帥', '郎平', '安卓', '卡拉黑', '卡拉斯科', 'S10', '瑞和寶', '世界杯', '小詹皇', '浦發', '宋鴻兵', '海王星', '高傭', '億航', '榮威', '姚明', '威廉姆斯', '上港', '李暉', '劉國梁', '曼聯', '小明', '奧恰洛夫', '唐汪鎮', '微博', '穆里尼奧', '李秋平', '羅傑斯', '萊萬特', '李大霄', '大神', '清華', '網易', '老馬', '魯能', '火箭隊', '皇馬', '芒格', '魅族', '重渡溝', '萊萬', '大聶', '鍾馗', '東決', '余則成', '小智', '尤納斯', '劉集', '韋德', '大智慧', '周評', '湯米', '本澤馬', '施泰納', '雷霆隊', '雅桑克萊', '陳柯楓', '梅西', '小強', '單薇恩', '吉尼斯', '陰陽師', '凱塔', '馬克思', '拳皇', '富力', '王者榮', '周雲傑', '朱', '精准', '國四', 'S11']
Na : ['門將', '秸稈', '棋牌', '節奏', '男乒', '系列', '紅糖', '農村人', '貨源', '潛質', '湖怪', '時間', '大麻', '用藥', '胖子', '中甲', '濃眉', '東西', '顛覆性', '困難戶', '年費', '魔王', '遊客', '神跡', '套路貸', '小辣椒', '綠道', '進展', '金融', '樂趣', '風潮', '球迷', '小額', '毒', '產品', '素顏', '召喚獸', '中單', '鄰居', '規則', '獨生子女', '好兄弟', '技能', '可口可樂', '總決賽', '人生', '托單', '白鰱', '洲際賽', '商標', '海島', '武器', '環保', '炭',

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [51]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = ['COLONCATEGORY', 'PAUSECATEGORY', 'WHITESPACE', 'COMMACATEGORY', 'QUESTIONCATEGORY', 'EXCLAMATIONCATEGORY',
    'FW', 'T', 'DASHCATEGORY', 'I', 'PARENTHESISCATEGORY', 'PERIODCATEGORY', 'Cab', 'SEMICOLONCATEGORY', 'ETCCATEGORY',
    'DOTCATEGORY']
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

In [52]:
get_bow_vector(train_title_cuts[100], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# Cosine Similarity

In [53]:
def cosine_similarity(bow1, bow2):
    eps: float=1e-8
    nbow1 = bow1 / (np.sqrt(np.sum(bow1**2)) + eps)
    nbow2 = bow2 / (np.sqrt(np.sum(bow2**2)) + eps)
    similarity = np.dot(nbow1,nbow2)
    return similarity

In [54]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882746415753

In [57]:
train_title_cuts = dict(zip(range(len(train_title_cuts)),train_title_cuts))
test_title_cuts = dict(zip(range(1800, len(test_title_cuts)+1800),test_title_cuts))

# Group mean vector

In [58]:
#all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']
group_vectors = {news_class: [] for news_class in all_news_class}
i = 0
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.00666667, 0.        , 0.        , ..., 0.00333333, 0.        ,
        0.01      ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0.        , 0.        , 0.00333333, ..., 0.        , 0.        ,
        0.        ]),
 '農業': array([0.        , 0.00333333, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 '遊戲': array([0., 0., 0., ..., 0., 0., 0.])}

# Group mean vector: 測試

In [59]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val 
    classification[max_class].append(index)

In [60]:
from collections import Counter
test = []
for group, ids in classification.items(): 
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 66, '遊戲': 9, '旅遊': 8, '財經': 7, '農業': 6, '科技': 4})
predict 財經 :  Counter({'財經': 66, '科技': 21, '農業': 11, '體育': 9, '旅遊': 9, '遊戲': 7})
predict 科技 :  Counter({'科技': 58, '財經': 14, '農業': 12, '體育': 11, '旅遊': 7, '遊戲': 4})
predict 旅遊 :  Counter({'旅遊': 62, '農業': 11, '財經': 9, '遊戲': 7, '體育': 5, '科技': 4})
predict 農業 :  Counter({'農業': 57, '旅遊': 6, '遊戲': 4, '體育': 2, '科技': 2, '財經': 1})
predict 遊戲 :  Counter({'遊戲': 69, '科技': 11, '旅遊': 8, '體育': 7, '財經': 3, '農業': 3})
